In [1]:
#importing libraries
import requests,zipfile, io, os, csv
import xml.etree.ElementTree as ET
from datetime import datetime
import pandas as pd
import boto3

In [2]:
today = str(datetime.today().date())

In [4]:
#opening log file
log = open(f'./log/log-{today}.txt','a+')

In [5]:
file_path = './case_study1_link.xml'

In [6]:
tree = ET.parse(file_path)

In [7]:
root = tree.getroot()

In [8]:
# downloaded first link for the file type FULINS
for item in root.iter('doc'):
    dict_attributes = {}
    for child in item.iter('str'):
        dict_attributes[child.attrib['name']] = child.text
    if dict_attributes['file_type'] == 'FULINS':
        response = requests.get(dict_attributes['download_link'])
        log.write(f'The response status code is {str(response.status_code)}\n')
        log.write(f"The download link is {str(dict_attributes['download_link'])}\n")
        log.write(f"The file type for the link is {str(dict_attributes['file_type'])}\n")
        zipFile = zipfile.ZipFile(io.BytesIO(response.content))
        zipFile.extractall("./downloaded_xml")
    break

In [9]:
for file in os.listdir("./downloaded_xml"):
    if 'xml' in file:
        xml_file_path = "./downloaded_xml/"+file
        tree1 = ET.parse(xml_file_path)

In [10]:
root1 = tree1.getroot()
# root1.tag

In [11]:
#acquiring namespace
my_namespaces = dict([node for _, node in ET.iterparse(xml_file_path, events = ['start-ns'])])

In [12]:
log.write(f'The namespace is {my_namespaces}\n')

131

In [13]:
#parsing the downloaded xml to get the required attributes

with open('Out.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csvwriter = csv.writer(csv_file)
    col_names = ['FinInstrmGnlAttrbts.Id','FinInstrmGnlAttrbts.FullNm','FinInstrmGnlAttrbts.ClssfctnTp','FinInstrmGnlAttrbts.CmmdtyDerivInd',
             'FinInstrmGnlAttrbts.NtnlCcy','Issr']
    csvwriter.writerow(col_names)

    for item in root1.iterfind('.//RefData',my_namespaces):
        event = []
        event.append(item.find('./FinInstrmGnlAttrbts/Id',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/FullNm',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/ClssfctnTp',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/CmmdtyDerivInd',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/NtnlCcy',my_namespaces).text)
        event.append(item.find('./Issr',my_namespaces).text)
        csvwriter.writerow(event)
        # print(item.find('Issr',my_namespaces).text,item.find('./FinInstrmGnlAttrbts/Id',my_namespaces).text,item.find('./Issr',my_namespaces).text)

In [14]:
df = pd.read_csv('Out.csv')

In [15]:
log.write(f'Rows extracted are {df.shape[0]}\n')

25

In [16]:
# dump the csv to the s3 bucket
s3 = boto3.resource(
    's3',
    aws_access_key_id='YOUR_ACCESS_KEY',
    aws_secret_access_key='YOUR_SECRET_KEY'
)
# Set up S3 bucket and file name
bucket_name = 'your-bucket-name'
file_name = 'your-file-name.csv'
# Set up CSV data
with open('Out.csv','rb') as file:
    data = file
# Write CSV file to S3
s3.Object(bucket_name, file_name).put(Body=data)

In [16]:
log.write(f'*****************************\n\n')
log.close()